In [36]:
import numpy as np
import pandas as pd
import sys
import json
import pickle5 as pickle
import nltk
import scispacy
import spacy
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
nltk.download('stopwords')
nltk.download("punkt")

from rank_bm25 import BM25Okapi

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ahmetsenturk/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ahmetsenturk/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [38]:
topics_file = open("topics.pickle", "rb")
all_documents_file = open("all_documents.pickle", "rb")
nlp = spacy.load("en_core_sci_md")

In [39]:
topics_dict = pickle.load(topics_file)
all_docs_dict = pickle.load(all_documents_file)

In [40]:
topics_df = pd.DataFrame(topics_dict)
all_docs_df = pd.DataFrame(all_docs_dict)

In [41]:
all_docs_df.drop_duplicates(subset=['document_id'], keep=False, inplace=True)

In [42]:
for index, topic in topics_df.iterrows():
    nlp_query = nlp(topic["query"]).ents
    nlp_question = nlp(topic["question"]).ents
    nlp_narrative = nlp(topic["narrative"]).ents
    
    resulting_query = ""
    resulting_question = ""
    resulting_narrative = ""
    
    for query_entites in nlp_query:  
        resulting_query = resulting_query + " " + str(query_entites).replace("\n", " ")
        
    for question_entites in nlp_question:
        resulting_question = resulting_question + " " + str(question_entites).replace("\n", " ")
       
    for narrative_entites in nlp_narrative:
        resulting_narrative = resulting_narrative + " " + str(narrative_entites).replace("\n", " ")
        
    topics_df.iloc[index]["query"] = topics_df.iloc[index]["query"] + " " + resulting_query
    topics_df.iloc[index]["question"] = topics_df.iloc[index]["question"] + " " + resulting_question
    topics_df.iloc[index]["narrative"] = topics_df.iloc[index]["narrative"] + " " + resulting_narrative

In [ ]:
for index, document in all_docs_df.iterrows():
    nlp_abstract = nlp(document["abstract"]).ents
    nlp_title = nlp(document["title"]).ents
    
    resulting_abstract = ""
    resulting_title = ""
    
    for abstract_entites in nlp_query:  
        resulting_abstract = resulting_abstract + " " + str(abstract_entites).replace("\n", " ")
        
    for title_entites in nlp_question:
        resulting_title = resulting_title + " " + str(title_entites).replace("\n", " ")
         
    all_docs_df.iloc[index]["abstract"] = all_docs_df.iloc[index]["abstract"] + " " + resulting_abstract
    all_docs_df.iloc[index]["title"] = all_docs_df.iloc[index]["title"] + " " + resulting_title

In [ ]:
all_docs_df["text"] = all_docs_df["abstract"] + " " + all_docs_df["title"] 

In [20]:
stop_words = set(stopwords.words('english')) 
tokenizer = RegexpTokenizer(r'\w+')
all_docs_splitted = []
for text in all_docs_df["text"]:
    if text != "":
        filtered_text = []
        for t in tokenizer.tokenize(text):
            if t not in stop_words:
                filtered_text.append(t.lower())
        all_docs_splitted.append(filtered_text)

In [21]:
bm25 = BM25Okapi(all_docs_splitted)

In [32]:
results = pd.DataFrame()
train_results = pd.DataFrame()
test_results = pd.DataFrame()
for topic_index in range(0, len(topics_df.index)):
    query = topics_df.iloc[topic_index]["query"] + " " + topics_df.iloc[topic_index]["question"] + " " + topics_df.iloc[topic_index]["narrative"]
    filtered_query = []
    if query != "":
        for t in tokenizer.tokenize(query):
            if t not in stop_words:
                filtered_query.append(t.lower())
    
    query_results = bm25.get_scores(filtered_query)
    query_results_df = pd.DataFrame(query_results)
    query_results_df.columns = ["bm25 results"]
    query_results_df.insert(0, "topic_id", [(topic_index + 1) for x in range(0, len(query_results_df.index))])
    query_results_df.insert(1, "temp1", ["Q0" for x in range(0, len(query_results_df.index))])
    query_results_df.insert(2, "doc_id", [all_docs_df.iloc[i,0] for i in range(0, len(query_results_df.index))])
    query_results_df.insert(3, "rank", ["0" for x in range(0, len(query_results_df.index))])
    query_results_df["standard"] = "STANDARD"
    results = pd.concat([results, query_results_df], axis=0) 
    if topic_index % 2 == 0:
        test_results = pd.concat([train_results, query_results_df], axis=0) 
    else:
        train_results = pd.concat([test_results, query_results_df], axis=0)    
    

In [29]:
np.savetxt("./results.txt", results.values, fmt="%s")

In [34]:
np.savetxt("./test_results.txt", test_results.values, fmt="%s")

In [ ]:
np.savetxt("./train_results.txt", train_results.values, fmt="%s")